<p style="font-weight:bold;"> <span style="font-size: 36px"> In Memory Initialization of unconfigured DataSource </span> </p>

<a id='init-to-memory'></a>

In [0]:
#!eval-notebook "InitSystemorphRefDataToMemory"

In [0]:
using Systemorph.Vertex.DataSource;

# Files by Data Node

In [0]:
var DataNodeFiles = new Dictionary<string,(string, string)[]>(){{"ES", new (string, string)[]{("DataNodes/DataNodes_ES.csv", ImportFormats.DataNode), 
                                                                        ("DataNodes/DataNodeStates_ES_2020_12.csv", ImportFormats.DataNodeState), 
                                                                        ("DataNodes/DataNodeParameters_ES_2020_12.csv", ImportFormats.DataNodeParameter)}}, 
                                                        {"FR", new (string, string)[]{("DataNodes/DataNodes_FR.csv", ImportFormats.DataNode), 
                                                                        ("DataNodes/DataNodeStates_FR_2020_12.csv", ImportFormats.DataNodeState), 
                                                                        ("DataNodes/DataNodeParameters_FR_2020_12.csv", ImportFormats.DataNodeParameter)}}, 
                                                        {"CH", new (string, string)[]{("DataNodes/DataNodes_CH.csv", ImportFormats.DataNode), 
                                                                        ("DataNodes/DataNodeStates_CH_2020_12.csv", ImportFormats.DataNodeState),  
                                                                        ("DataNodes/DataNodeParameters_CH_2020_12.csv", ImportFormats.DataNodeParameter)}}};

In [0]:
var TransactionalFilesByDataNode = new Dictionary<string, (string, string)[]>(){{"ES", new (string, string)[]{("TransactionalData/Openings_ES_2020_12.csv", ImportFormats.Opening), 
                                                                                        ("TransactionalData/NominalCashflows_ES_2020_12.csv", ImportFormats.Cashflow), 
                                                                                        ("TransactionalData/Actuals_ES_2020_12.csv", ImportFormats.Actual)}}, 
                                                                    {"FR", new (string, string)[]{("TransactionalData/Openings_FR_2020_12.csv", ImportFormats.Opening), 
                                                                                        ("TransactionalData/NominalCashflows_FR_2020_12.csv", ImportFormats.Cashflow), 
                                                                                        ("TransactionalData/Actuals_FR_2020_12.csv", ImportFormats.Actual)}}, 
                                                                    {"CH", new (string, string)[]{("TransactionalData/Openings_CH_2020_12.csv", ImportFormats.Opening), 
                                                                                        ("TransactionalData/NominalCashflows_CH_2020_12.csv", ImportFormats.Cashflow),  
                                                                                        ("TransactionalData/Actuals_CH_2020_12.csv", ImportFormats.Actual), 
                                                                                        ("TransactionalData/SimpleValue_CH_2020_12.csv", ImportFormats.SimpleValue), 
                                                                                        ("TransactionalData/NominalCashflows_CH_2020_12_MTUP10pct.csv", ImportFormats.Cashflow), 
                                                                                        ("TransactionalData/NominalCashflows_CH_2021_3.csv", ImportFormats.Cashflow), 
                                                                                        ("TransactionalData/Actuals_CH_2021_3.csv", ImportFormats.Actual)}}};

# Import Tasks

In [0]:
public async Task InitializeDataNodesAsync(IDataSourceVariable dataSource, string pathToImport, string[] nodes = null){
    nodes = nodes ?? new string[]{"ES", "FR", "CH"};
    var uploads = DataNodeFiles.Where(kvp => nodes.Contains(kvp.Key)).SelectMany(kvp => kvp.Value).ToArray();
    foreach(var pair in uploads) 
        await Import.FromFile(pathToImport + pair.Item1).WithFormat(pair.Item2).WithTarget(dataSource).WithActivityLog().ExecuteAsync();
}

In [0]:
public async Task InitializeDataNodesAsync(IWorkspace workspace, string pathToImport, string[] nodes = null){
    nodes = nodes ?? new string[]{"ES", "FR", "CH"};
    var uploads = DataNodeFiles.Where(kvp => nodes.Contains(kvp.Key)).SelectMany(kvp => kvp.Value).ToArray();
    foreach(var pair in uploads)
        await Import.FromFile(pathToImport + pair.Item1).WithFormat(pair.Item2).WithTarget(workspace).WithActivityLog().ExecuteAsync();
}

In [0]:
public async Task InitializeTransactionalDataAsync(IDataSourceVariable dataSource, string pathToImport, string[] nodes = null){
    nodes = nodes ?? new string[]{"ES", "FR", "CH"};
    var uploads = TransactionalFilesByDataNode.Where(kvp => nodes.Contains(kvp.Key)).SelectMany(kvp => kvp.Value).ToArray();
    foreach(var pair in uploads)
        await Import.FromFile(pathToImport + pair.Item1).WithFormat(pair.Item2).WithTarget(dataSource).WithActivityLog().ExecuteAsync();
}

In [0]:
public async Task InitializeTransactionalDataAsync(IWorkspace workspace, string pathToImport, string[] nodes = null){
    nodes = nodes ?? new string[]{"ES", "FR", "CH"};
    var uploads = TransactionalFilesByDataNode.Where(kvp => nodes.Contains(kvp.Key)).SelectMany(kvp => kvp.Value).ToArray();
    foreach(var pair in uploads)
        await Import.FromFile(pathToImport + pair.Item1).WithFormat(pair.Item2).WithTarget(workspace).WithActivityLog().ExecuteAsync();
}

In [0]:
//await InitializeDataNodesAsync();

# Import of Transactional Data

## FR Transactional Data

In [0]:
/* await Import.FromFile(pathToImport + "TransactionalData/NominalCashflows_FR_2020_12.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

In [0]:
/* await Import.FromFile(pathToImport + "TransactionalData/Actuals_FR_2020_12.csv")
            .WithFormat(ImportFormats.Actual)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

In [0]:
//DataSource.Partition.GetCurrentPartitions()

## ES Transactional Data

In [0]:
/* await Import.FromFile(pathToImport + "TransactionalData/Openings_ES_2020_12.csv")
            .WithFormat(ImportFormats.Opening)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

In [0]:
/* await Import.FromFile(pathToImport + "TransactionalData/NominalCashflows_ES_2020_12.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

In [0]:
/* await Import.FromFile(pathToImport + "TransactionalData/Actuals_ES_2020_12.csv")
            .WithFormat(ImportFormats.Actual)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

In [0]:
//DataSource.Partition.GetCurrentPartitions()

## CH Transactional Data

In [0]:
/*await Import.FromFile(pathToImport + "TransactionalData/Openings_CH_2020_12.csv")
            .WithFormat(ImportFormats.Opening)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()*/

In [0]:
/*await Import.FromFile(pathToImport + "TransactionalData/NominalCashflows_CH_2020_12.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()*/

In [0]:
/*await Import.FromFile(pathToImport + "TransactionalData/Actuals_CH_2020_12.csv")
            .WithFormat(ImportFormats.Actual)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

In [0]:
/*await Import.FromFile(pathToImport + "TransactionalData/SimpleValue_CH_2020_12.csv")
            .WithFormat(ImportFormats.SimpleValue )
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

In [0]:
/*await Import.FromFile(pathToImport + "TransactionalData/NominalCashflows_CH_2020_12_MTUP10pct.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()*/

In [0]:
/*await Import.FromFile(pathToImport + "TransactionalData/NominalCashflows_CH_2021_3.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()*/

In [0]:
/*await Import.FromFile(pathToImport + "TransactionalData/Actuals_CH_2021_3.csv")
            .WithFormat(ImportFormats.Actual)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync() */

# Reset Workspace

In [0]:
//Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [0]:
//DataSource.Partition.GetCurrentPartitions()